# 12-Class SSVEP Dataset
## Classification Using Canonical Correaltion Analysis (CCA)
### The following is implemented on a 12-Class publicly available SSVEP EEG Dataset

<img src="12_classSSVEP.png">

#### Dataset URL: 
https://github.com/mnakanishi/12JFPM_SSVEP/tree/master/data

#### Dataset Paper:
Masaki Nakanishi, Yijun Wang, Yu-Te Wang and Tzyy-Ping Jung, 
"A Comparison Study of Canonical Correlation Analysis Based Methods for Detecting Steady-State Visual Evoked Potentials," 
PLoS One, vol.10, no.10, e140703, 2015. 

#### Implementation:
Note: Following implementation is an asynchronous SSVEP BCI using CCA classification for 1 second data length

Aravind Ravi | eBionics Lab | University of Waterloo

In [38]:
import scipy.io as sio
from scipy.signal import butter, lfilter, filtfilt
from scipy.signal import freqz
from sklearn.cross_decomposition import CCA
from sklearn.metrics import confusion_matrix
import numpy as np
import math

#### buffer() - Function to segment the data based on fixed window lengths and overlap (Windowing)

In [39]:
def buffer(data,duration,dataOverlap):
    numberOfSegments = int(math.ceil((len(data)-dataOverlap)/(duration-dataOverlap)))
    #print(data.shape)
    tempBuf = [data[i:i+duration] for i in range(0,len(data),(duration-int(dataOverlap)))]
    tempBuf[numberOfSegments-1] = np.pad(tempBuf[numberOfSegments-1],(0,duration-tempBuf[numberOfSegments-1].shape[0]),'constant')
    tempBuf2 = np.vstack(tempBuf[0:numberOfSegments])
    return tempBuf2

#### butterBandpass () - Function generates the filter coefficients
#### butterBandpassFilter () - Function performs filtering on the input data


In [40]:
def butterBandpass(lowcut, highcut, fs, order=4):
    nyq = 0.5 * fs
    low = lowcut / nyq
    high = highcut / nyq
    b, a = butter(order, [low, high], btype='band')
    return b, a

def butterBandpassFilter(data, lowcut, highcut, fs, order=4):
    b, a = butterBandpass(lowcut, highcut, fs, order=order)
    y = lfilter(b, a, data)
    return y

#### Canonical Correlation Analysis (CCA)
$$\DeclareMathOperator*{\argmax}{argmax}$$

Consider two multidimensional variables $X$, $Y$ where $X$ refers to the set of multi-channel EEG data and $Y$ refers to the set of reference signals of the same length as $X$. The linear combinations of $X$ and $Y$ are given as $x = X'W_{x}$ and $y = Y'W_{y}$. CCA finds the weights, $W_{x}$ and $W_{y}$ that maximize the correlation between $x$ and $y$ by solving (1). The maximum of $\rho$ with respect to $W_{x}$ and $W_{y}$ is the maximum correlation.

$$\max_{W_{x},W_{y}} \rho(x,y) = \frac{\mathbb{E}{[W_{x}'XY'W_{y}]}}{\sqrt{\mathbb{E}{[W_{x}'XX'W_{x}]}\mathbb{E}{[W_{y}'YY'W_{y}]}}}$$

The reference signals $Y_{n}$  are defined as:

$$Y_{n} = \begin{bmatrix} \sin({2 \pi f_{n}t}) \\ \cos({2 \pi f_{n}t}) \\ \vdots \\ \sin({4 \pi  f_{n}t}) \\ \cos({4 \pi  f_{n}t}) \end{bmatrix},t = \begin{bmatrix} 
    \frac{1}{f_{s}}
    \frac{2}{f_{s}}
    \dots
    \frac{N_{s}}{f_{s}}
    \end{bmatrix}$$
    
where $Y_{n} \in \mathbb{R}^{2 N_{h} \times N_{s}} $, $f_{n}$ is the stimulation frequency, $f_{s}$ is the sampling frequency, $N_{s}$ is number of samples, and $N_{h}$ is the number of harmonics. Here, $N_{h}=2$. The canonical correlation features $\rho_{f_{i}}$, where $i = 1,2,...,7$ are extracted for each segment of the EEG data, and the output class $C$ for a given sample can be determined as: $C = \argmax (\rho_{f_{i}})$

#### getReferenceSignals () - Function generates sinusoidal reference templates for CCA for the first and second harmonics
#### findCorr () - Function performs CCA on input EEG and set of reference sinusoids

In [41]:
def getReferenceSignals(length, target_freq,samplingRate):
    
    reference_signals = []
    t = np.arange(0, (length/(samplingRate)), step=1.0/(samplingRate))
    #First harmonics/Fundamental freqeuncy
    reference_signals.append(np.sin(np.pi*2*target_freq*t))
    reference_signals.append(np.cos(np.pi*2*target_freq*t))
    #Second harmonics
    reference_signals.append(np.sin(np.pi*4*target_freq*t))
    reference_signals.append(np.cos(np.pi*4*target_freq*t))
    reference_signals = np.array(reference_signals)
    return reference_signals

def findCorr(n_components,numpy_buffer,freq):
    # Perform Canonical correlation analysis (CCA)
    # numpy_buffer - consists of the EEG
    # freq - set of sinusoidal reference templates corresponding to the flicker frequency
    cca = CCA(n_components)
    corr=np.zeros(n_components)
    result=np.zeros(freq.shape[0])
    for freq_idx in range(0,freq.shape[0]):
        cca.fit(numpy_buffer.T,np.squeeze(freq[freq_idx,:,:]).T)
        O1_a,O1_b = cca.transform(numpy_buffer.T, np.squeeze(freq[freq_idx,:,:]).T)
        ind_val=0
        for ind_val in range(0,n_components):
            corr[ind_val] = np.corrcoef(O1_a[:,ind_val],O1_b[:,ind_val])[0,1]
            result[freq_idx] = np.max(corr)
    return result

In [42]:
#Variable to store all accuracies
all_acc=np.zeros((10,1))

#Iterate through all 10 subjects 
for sub in range(0,10):

#Load the EEG Dataset
    dataset=sio.loadmat('s'+str(sub+1)+'.mat')
   
    eeg=np.array(dataset['eeg'],dtype='float32')

#Reading the required parameters
#Number of classes	
    num_classes=eeg.shape[0]
#Number of EEG channels
    num_chan=eeg.shape[1]
#Trial length of EEG
    trial_len=eeg.shape[2]
#Total number of trials
    num_trials=eeg.shape[3]
    sample_rate = 256
#SSVEP flicker frequencies used for the 12 SSVEP targets   
    flick_freq=np.array([9.25,11.25,13.25,9.75,11.75,13.75,10.25,12.25,14.25,10.75,12.75,14.75])
#variable to store the true labels    
    labels=[]
    filtered_data = np.zeros(eeg.shape)
#Iterate through the channels and trials for filtering the data using Bandpass filter between 6 Hz and 80 Hz
    for cl in range(0,num_classes):
        for ch in range(0,num_chan):
            for tr in range(0,num_trials):
                filtered_data[cl,ch,:,tr] = butterBandpassFilter(eeg[cl,ch,:,tr],6,80,sample_rate,order=4)

#Extract the actual trial from the data (refer the paper for more details)
    filtered_data = filtered_data[:,:,int(38+0.135*sample_rate):int(38+0.135*sample_rate+4*sample_rate-1),:]
    eeg=[]
#Segment the EEG trials into 1 second non-overlapping segments 
#Window/segment length in seconds
    window_len=1
#Shift of the window in seconds (indirectly specifies overlap)
    shift_len=1

#Converting into units of samples
    duration=int(window_len*sample_rate)
    data_overlap = (window_len-shift_len)*sample_rate
    
    numberOfSegments = int(math.ceil((filtered_data.shape[2]-data_overlap)/(duration-data_overlap)))
    
    segmented_data = np.zeros((filtered_data.shape[0],filtered_data.shape[1],numberOfSegments,duration,filtered_data.shape[3]))

#Performing data segmentation on each trial and each channel for all classes of data
    for cl in range(0,num_classes):
        for ch in range(0,num_chan):
            for tr in range(0,num_trials):
                segmented_data[cl,ch,:,:,tr]=buffer(filtered_data[cl,ch,:,tr],duration,data_overlap)
    
    filtered_data=[]
    freq_ref=[]
#Generating the required sinusoidal templates for the given 12-class SSVEP classification
    for fr in range(0,len(flick_freq)):
        freq_ref.append(getReferenceSignals(duration,flick_freq[fr],sample_rate))
    
    freq_ref=np.array(freq_ref,dtype='float32')
    
    predicted_class=[]
#Performing segment wise classification using CCA 
    for cl in range(0,num_classes):
        for tr in range(0,num_trials):
            for sg in range(0,numberOfSegments):
#True labels are created here
                labels.append(cl)
#Perform CCA on a single segment
                result=findCorr(1,segmented_data[cl,:,sg,:,tr],freqRef)
#Pick the class that corresponds to the maximum canonical correlation coefficient
                predicted_class.append(np.argmax(result)+1)
    
    segmented_data=[]
    labels=np.array(labels)+1
    predicted_class=np.array(predicted_class)
#creating a confusion matrix of true versus predicted classification labels
    c_mat = confusion_matrix(labels, predicted_class)
#computing the accuracy from the confusion matrix
    accuracy=np.divide(np.trace(c_mat),np.sum(np.sum(c_mat)))
    all_acc[sub] = accuracy
    print("Subject:",sub+1, " - Accuracy:",accuracy*100,"%")
#Mean overall accuracy across all subjects
print("Overall Accuracy Across Subjects:",np.mean(all_acc)*100,"%","std:",np.std(all_acc)*100,"%")

Subject: 1  - Accuracy: 28.333333333333332 %
Subject: 2  - Accuracy: 26.666666666666668 %
Subject: 3  - Accuracy: 61.25000000000001 %
Subject: 4  - Accuracy: 79.58333333333333 %
Subject: 5  - Accuracy: 42.916666666666664 %
Subject: 6  - Accuracy: 86.25 %
Subject: 7  - Accuracy: 66.38888888888889 %
Subject: 8  - Accuracy: 95.83333333333334 %
Subject: 9  - Accuracy: 67.77777777777779 %
Subject: 10  - Accuracy: 65.27777777777779 %
Overall Accuracy Across Subjects: 62.02777777777777 % std: 22.027112221975585 %
